In [67]:
from colorama import Fore, init
init(autoreset=True)
import copy
import math
import time
from numpy import dtype
from queue import PriorityQueue
import numpy as np
from IPython.display import Markdown, display

Definim modelarea problemei printr-o matrice de tupluri de 3x3. Pentru modelare definim 2 functii auxiliare pentru transformarea listei initiala in matrice si pentru transformarea matricei inpoi in lista: 

In [52]:
def model_problem(array):
    matrice = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    for i in range(9):
        matrice[i // 3][i % 3] = (array[i], False)
        if array[i] == 0:
            i_zero = i // 3
            j_zero = i % 3
    directii = [(i_zero - 1, j_zero), (i_zero + 1, j_zero), (i_zero, j_zero - 1), (i_zero, j_zero + 1)]
    for i, j in directii:
        if 0 <= i < 3 and 0 <= j < 3:
            matrice[i][j] = (matrice[i][j][0], True)
    return matrice

def model_back(matrice):
    a = [0,0,0,0,0,0,0,0,0]
    for i in range(3):
        for j in range(3):
            a[i*3+j] = matrice[i][j][0]
    return a


#Teste pentru functiile auxiliare
initial_array = [1, 5, 2, 7, 0, 4, 8, 6, 3] 
matrix = model_problem(initial_array)
print(model_problem(initial_array))
print(model_back(matrix))


[[(1, False), (5, True), (2, False)], [(7, True), (0, False), (4, True)], [(8, False), (6, True), (3, False)]]
[1, 5, 2, 7, 0, 4, 8, 6, 3]


Pe langa acestea am definit 2 functii de printare a unei instante a jocului:

In [53]:
def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end='\t')
        print() 

def print_game(matrix):
    output = ""
    for row in matrix:
        output += '|'
        for element in row:
            if element[1]:
                color = f'<span style="color:green">{element[0]}</span>'
                output += f"{color} | "
            elif element[0] == 0:
                output += '0 | '
            else:
                color = f'<span style="color:red">{element[0]}</span>'
                output += f"{color} | "
        output += "<br>"  # Add line break for the next row
    display(Markdown(output))


print_matrix(matrix)
print_game(matrix)


(1, False)	(5, True)	(2, False)	
(7, True)	(0, False)	(4, True)	
(8, False)	(6, True)	(3, False)	


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

Tot ca functii auxiliare am definit 2 functii meniite sa gaseasca pozitia unui numar in joc respectiv pozitia celulei goale din joc

In [54]:
# returneaza pozitia valorii n din matricea de joc sub forma de tuplu (i,j)
def find_positions(matrice, n):
    for i in range(3):
        for j in range(3):
            if matrice[i][j][0] == n:
                return (i, j)
                      
# returneaza pozitia valorii 0 din matricea de joc
def find_zero(matrice):
    for i in range(3):
        for j in range(3):
            if matrice[i][j][0] == 0:
                return (i, j)
print(find_positions(matrix,6))
print(find_zero(matrix))

(2, 1)
(1, 1)


Identificarea starilor speciale se face cu functiile verificare_stare_initiala si verificare_stare_finala

In [55]:
def verificare_stare_finala(matrice):
    flattened_matrix = [matrice[i][j][0] for i in range(3) for j in range(3)]
    flattened_matrix.remove(0)
    return flattened_matrix == sorted(flattened_matrix)

def verificare_stare_initiala(matrice):
    array = model_back(matrice)
    if array == initial_array:
        return True
    else:
        return False


Tranzitiile sunt implementate cu ajutorul functiei de tranzitie care are ca parametrii matricea in care dorim sa jucam si numarul celulei de mutat. Functia de tranzitie se foloseste de o functie de validare a tranzitiei (care verifica daca flagul celulei de mutat e True) iar apoi muta celula cu numarul primit ca paramentru in spatiul liber unic in joc.

In [56]:
# returneaza valoarea flag-ului (true/false) pt valoarea n
def validare_tranzitie(matrice, n):
    return matrice[find_positions(matrice, n)[0]][find_positions(matrice, n)[1]][1]

# functie care face tranzitia nr n in locul null
def tranzitie(matrice, n):
    m = copy.deepcopy(matrice)
    i, j = find_positions(matrice, n)
    i0, j0 = find_zero(matrice)
    if matrice[i][j][1]:
        # actualizarea flag-urilor pt vechile valor care nu mai pot si mutate
        directii = [(i0 - 1, j0), (i0 + 1, j0), (i0, j0 - 1), (i0, j0 + 1)]
        for ii, jj in directii:
            if 0 <= ii < 3 and 0 <= jj < 3 and m[ii][jj][0] != n:
                m[ii][jj] = (m[ii][jj][0], False)

        # actualizare flag-uri pt noile valori care pot fi mutate (vecinii lui n)
        directii = [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
        for ii, jj in directii:
            if 0 <= ii < 3 and 0 <= jj < 3 and m[ii][jj][0] != 0:
                m[ii][jj] = (m[ii][jj][0], True)
        # actualizarea pozitie pt n & null
        m[i0][j0] = (n, False)
        m[i][j] = (0, False)
        return m

    else:
        print("tranzitie nepermisa")
        print_game(m)
        print(n)

m = model_problem( [8, 6, 7, 2, 5, 4, 0, 3, 1])
print_game(m)
x = tranzitie(m, 3)
print("Matrice dupa ce apelam tranzitie(matrice, 3):")
print_game(x)

|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

Matrice dupa ce apelam tranzitie(matrice, 3):


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:red">2</span> | <span style="color:green">5</span> | <span style="color:red">4</span> | <br>|<span style="color:red">3</span> | 0 | <span style="color:green">1</span> | <br>

In final definim functia de IDDFS search care itereaza toate stagiile posibile in care poate ajunge jocul intr-un numar maxim de miscari si returneaza sirul de miscari necesare pentru a ajunge in starea finala prin cel mai mic numar de pasi.

In [57]:
def IDDFS_search(matrice, maxSteps):
    nthStepDict = {tuple(tuple(row) for row in matrice): []}
    step = 0

    while step <= maxSteps:
        nextStepDict = {}
        step += 1

        for m, moveList in nthStepDict.items():
            for cell in  [m[i][j][0] for i in range(3) for j in range(3) if m[i][j][1]]:
                moveMatrix  = tranzitie(list(list(row) for row in m), cell)

                if verificare_stare_finala(moveMatrix):
                    return moveList + [cell]
                
                nextStepDict[tuple(tuple(row) for row in moveMatrix )] = moveList + [cell]

        nthStepDict = nextStepDict

    return "Limita de pasi atinsa"

Testam codul pe cateva variante de joc generate si rezolavte de https://deniz.co/8-puzzle-solver/

In [58]:

stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
print_game(matrix)
print(IDDFS_search(matrix, 100))
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
print_game(matrix)
print(IDDFS_search(matrix, 100))

stateList = [8, 6, 7, 2, 5, 4, 0, 3, 1]
matrix = model_problem(stateList)
print_game(matrix)
print(IDDFS_search(matrix, 100))

stateList = [2, 7, 5, 0, 8, 4, 3, 1, 6]
matrix = model_problem(stateList)
print_game(matrix)
print(IDDFS_search(matrix, 100))


|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

[7, 5, 2, 1, 4]


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

[4, 3, 6, 8, 7, 4, 5, 2, 3, 6]


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

[2, 8, 6, 5, 3, 1, 4, 7, 5, 3, 1, 2, 8, 6, 3, 1, 2, 8, 6, 3, 1, 2, 7, 4, 8, 7, 4, 5]


|<span style="color:green">2</span> | <span style="color:red">7</span> | <span style="color:red">5</span> | <br>|0 | <span style="color:green">8</span> | <span style="color:red">4</span> | <br>|<span style="color:green">3</span> | <span style="color:red">1</span> | <span style="color:red">6</span> | <br>

[2, 7, 5, 4, 8, 1, 6, 8, 4, 5, 1, 2, 7, 1, 2, 7, 3, 6, 7, 4, 5]


Calculul miscarilor pe baza principiului greedy cu euristica Manhatten distance (suma dintre distanta fiecarie celule pana la pozitia in jocul sortat)

In [59]:
def manhattan_distance(matrix):
    distance = 0
    for i in range(3):
        for j in range(3):
            if matrix[i][j][0] != 0:
                x, y = (matrix[i][j][0] - 1) // 3, (matrix[i][j][0] - 1) % 3
                distance += abs(x - i) + abs(y - j)
    return distance

def greedy_manhattan(matrice, stepLimit):
    moves = []
    step = 0
    while not verificare_stare_finala(matrice) and step < stepLimit:
        step += 1
        min_distance = float('inf')
        cell = [matrice[i][j][0] for i in range(3) for j in range(3) if matrice[i][j][1]][-1]
        for c in [matrice[i][j][0] for i in range(3) for j in range(3) if matrice[i][j][1]]:
            distance = manhattan_distance(tranzitie(matrice, c))
            if distance < min_distance:
                reserve = cell
                min_distance = distance
                cell = c
        if len(moves) > 10:
            for i in range(2, 10):
                rep = []
                for _ in range(i):
                    rep += moves[-i:]
                if(moves[-i*i:] == rep and reserve != 0):
                    del moves[-i*i:]
                    cell = reserve
        matrice = tranzitie(matrice, cell)
        moves.append(cell)
    if step == stepLimit:
        return "Limita de pasi atinsa"
    else:
        return moves

In [60]:
stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_manhattan(matrix, 200))
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_manhattan(matrix, 200))

stateList = [8, 6, 7, 2, 5, 4, 0, 3, 1]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_manhattan(matrix, 200))

stateList = [2, 7, 5, 0, 8, 4, 3, 1, 6]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_manhattan(matrix, 200))

|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

[7, 5, 2, 1, 4]


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

[5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5]


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

[2, 8, 6, 7, 4, 1, 3, 2, 8, 6, 7, 4, 1, 3, 2, 8, 6, 7, 4, 1, 3, 2, 8, 6, 7, 4, 1, 3, 2, 5, 6, 8, 5, 6, 3, 2, 4, 6, 2, 1, 6, 4, 5, 8, 4, 6, 1, 2, 3, 5, 6, 4, 8, 6, 5, 3, 2, 1, 4, 5, 6]


|<span style="color:green">2</span> | <span style="color:red">7</span> | <span style="color:red">5</span> | <br>|0 | <span style="color:green">8</span> | <span style="color:red">4</span> | <br>|<span style="color:green">3</span> | <span style="color:red">1</span> | <span style="color:red">6</span> | <br>

Limita de pasi atinsa


Calculul miscarilor pe baza principiului greedy cu euristica Hamming distance(cate elemente sunt plasate gresit in raport cu starea finala)

In [61]:
def hamming_distance(matrix):
    distance = 0
    for i in range(3):
        for j in range(3):
            if matrix[i][j][0] != 0 and matrix[i][j][0] != 3 * i + j + 1: #se verifica daca elementul este pe pozitia corecta deja
                distance += 1
    return distance

def greedy_hamming(matrice, stepLimit):
    moves = []
    step = 0
    while not verificare_stare_finala(matrice) and step < stepLimit:
        step += 1
        min_distance = float('inf')
        cell = [matrice[i][j][0] for i in range(3) for j in range(3) if matrice[i][j][1]][-1]
        for c in [matrice[i][j][0] for i in range(3) for j in range(3) if matrice[i][j][1]]:
            distance = hamming_distance(tranzitie(matrice, c))
            if distance < min_distance:
                reserve = cell
                min_distance = distance
                cell = c
        if len(moves) > 10:
            for i in range(2, 10):
                rep = []
                for _ in range(i):
                    rep += moves[-i:]
                if (moves[-i * i:] == rep and reserve != 0):
                    del moves[-i * i:]
                    cell = reserve
        matrice = tranzitie(matrice, cell)
        moves.append(cell)

    if step == stepLimit:
        return "Limita de pasi atinsa"
    else:
        return moves
    

In [62]:
stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_hamming(matrix, 200))
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_hamming(matrix, 200))

stateList = [8, 6, 7, 2, 5, 4, 0, 3, 1]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_hamming(matrix, 200))

stateList = [2, 7, 5, 0, 8, 4, 3, 1, 6]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_hamming(matrix, 200))

|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

Limita de pasi atinsa


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

[5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5]


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

Limita de pasi atinsa


|<span style="color:green">2</span> | <span style="color:red">7</span> | <span style="color:red">5</span> | <br>|0 | <span style="color:green">8</span> | <span style="color:red">4</span> | <br>|<span style="color:green">3</span> | <span style="color:red">1</span> | <span style="color:red">6</span> | <br>

Limita de pasi atinsa


Calculul miscarilor pe baza principiului greedy cu euristica Euclideana (calculează distanța dintre pozițiile actuale ale elementelor și pozițiile corecte ale acestora utilizând formula distanței euclidiene)

In [63]:
def euclidean_distance(matrix):
    distance = 0
    for i in range(3):
        for j in range(3):
            if matrix[i][j][0] != 0:
                x, y = (matrix[i][j][0] - 1) // 3, (matrix[i][j][0] - 1) % 3
                dx = x - i
                dy = y - j
                distance += math.sqrt(dx**2 + dy**2)
    return distance

def greedy_euclidean(matrice, stepLimit):
    moves = []
    step = 0
    while not verificare_stare_finala(matrice) and step < stepLimit:
        step += 1
        min_distance = float('inf')
        cell = [matrice[i][j][0] for i in range(3) for j in range(3) if matrice[i][j][1]][-1]
        for c in [matrice[i][j][0] for i in range(3) for j in range(3) if matrice[i][j][1]]:
            distance = euclidean_distance(tranzitie(matrice, c))
            if distance < min_distance:
                reserve = cell
                min_distance = distance
                cell = c
        if len(moves) > 10:
            for i in range(2, 10):
                rep = []
                for _ in range(i):
                    rep += moves[-i:]
                if (moves[-i * i:] == rep and reserve != 0):
                    del moves[-i * i:]
                    cell = reserve
        matrice = tranzitie(matrice, cell)
        moves.append(cell)

    if step == stepLimit:
        return "Limita de pasi atinsa"
    else:
        return moves

In [64]:
stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_euclidean(matrix, 500))
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_euclidean(matrix, 500))

stateList = [8, 6, 7, 2, 5, 4, 0, 3, 1]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_euclidean(matrix, 500))

stateList = [2, 7, 5, 0, 8, 4, 3, 1, 6]
matrix = model_problem(stateList)
print_game(matrix)
print(greedy_euclidean(matrix, 500))

|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

Limita de pasi atinsa


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

[5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5]


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

[3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 5, 2, 8, 6, 2, 5, 4, 1, 3, 8, 6, 2, 5, 3, 8, 6, 2, 5, 3, 8, 6, 2, 5, 3, 8, 5, 3, 8, 5, 6, 2, 3, 6, 5]


|<span style="color:green">2</span> | <span style="color:red">7</span> | <span style="color:red">5</span> | <br>|0 | <span style="color:green">8</span> | <span style="color:red">4</span> | <br>|<span style="color:green">3</span> | <span style="color:red">1</span> | <span style="color:red">6</span> | <br>

Limita de pasi atinsa


#6 programul care ruleaza toate strategiile

In [65]:
def all_strategies(matrix):
    print_game(matrix)
    start_time = time.time()
    rez_IDDFS = IDDFS_search(matrix, 40)
    end_time = time.time()
    execution_time_IDDFS = end_time - start_time
    if(rez_IDDFS == "Limita de pasi atinsa"):
        print("Solution not found with IDDFS in a 40 move limit")
    else:
        print("Solution found with IDDFS: " + str(rez_IDDFS) + " with a length of : " + str(len(rez_IDDFS)))
    print("Execution time : " + str(execution_time_IDDFS) + " seconds")

    print()
    
    start_time = time.time()
    rez_Manhatten = greedy_manhattan(matrix, 200)
    end_time = time.time()
    execution_time_Manhatten = end_time - start_time
    if(rez_Manhatten == "Limita de pasi atinsa"):
        print("Solution not found with Manhatten heuristic in a 200 move limit")
    else:
        print("Solution found with Manhatten heuristic: " + str(rez_Manhatten) + " with a length of : " + str(len(rez_Manhatten)))
    print("Execution time : " + str(execution_time_Manhatten) + " seconds")

    print()

    start_time = time.time()
    rez_Hamming = greedy_hamming(matrix, 200)
    end_time = time.time()
    execution_time_Hamming = end_time - start_time
    if(rez_Hamming == "Limita de pasi atinsa"):
        print("Solution not found with Hamming heuristic in a 200 move limit")
    else:
        print("Solution found with Hamming heuristic: " + str(rez_Hamming) + " with a length  of : " + str(len(rez_Hamming)))
    print("Execution time : " + str(execution_time_Hamming) + " seconds")

    print()

    start_time = time.time()
    rez_Custom = greedy_euclidean(matrix, 200)
    end_time = time.time()
    execution_time_Custom = end_time - start_time
    if(rez_Custom == "Limita de pasi atinsa"):
        print("Solution not found with Custom heuristic in a 200 move limit")
    else:
        print("Solution found with Custom heuristic: " + str(rez_Custom) + " with a length of : " + str(len(rez_Custom)))
    print("Execution time : " + str(execution_time_Custom) + " seconds")


In [66]:
stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
all_strategies(matrix)
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
all_strategies(matrix)

stateList = [8, 6, 7, 2, 5, 4, 0, 3, 1]
matrix = model_problem(stateList)
all_strategies(matrix)

stateList = [2, 7, 5, 0, 8, 4, 3, 1, 6]
matrix = model_problem(stateList)
all_strategies(matrix)

|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

Solution found with IDDFS: [7, 5, 2, 1, 4] with a length of : 5
Execution time : 0.000997781753540039 seconds

Solution found with Manhatten heuristic: [7, 5, 2, 1, 4] with a length of : 5
Execution time : 0.0 seconds

Solution not found with Hamming heuristic in a 200 move limit
Execution time : 0.01750946044921875 seconds

Solution not found with Custom heuristic in a 200 move limit
Execution time : 0.02699756622314453 seconds


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

Solution found with IDDFS: [4, 3, 6, 8, 7, 4, 5, 2, 3, 6] with a length of : 10
Execution time : 0.0453343391418457 seconds

Solution found with Manhatten heuristic: [5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5] with a length of : 19
Execution time : 0.002064228057861328 seconds

Solution found with Hamming heuristic: [5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5] with a length  of : 19
Execution time : 0.0010023117065429688 seconds

Solution found with Custom heuristic: [5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5] with a length of : 19
Execution time : 0.0029668807983398438 seconds


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

Solution found with IDDFS: [2, 8, 6, 5, 3, 1, 4, 7, 5, 3, 1, 2, 8, 6, 3, 1, 2, 8, 6, 3, 1, 2, 7, 4, 8, 7, 4, 5] with a length of : 28
Execution time : 69.07343339920044 seconds

Solution found with Manhatten heuristic: [2, 8, 6, 7, 4, 1, 3, 2, 8, 6, 7, 4, 1, 3, 2, 8, 6, 7, 4, 1, 3, 2, 8, 6, 7, 4, 1, 3, 2, 5, 6, 8, 5, 6, 3, 2, 4, 6, 2, 1, 6, 4, 5, 8, 4, 6, 1, 2, 3, 5, 6, 4, 8, 6, 5, 3, 2, 1, 4, 5, 6] with a length of : 61
Execution time : 0.007004499435424805 seconds

Solution not found with Hamming heuristic in a 200 move limit
Execution time : 0.01803755760192871 seconds

Solution found with Custom heuristic: [3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 5, 2, 8, 6, 2, 5, 4, 1, 3, 8, 6, 2, 5, 3, 8, 6, 2, 5, 3, 8, 6, 2, 5, 3, 8, 5, 3, 8, 5, 6, 2, 3, 6, 5] with a length of : 65
Execution time : 0.007002592086791992 seconds


|<span style="color:green">2</span> | <span style="color:red">7</span> | <span style="color:red">5</span> | <br>|0 | <span style="color:green">8</span> | <span style="color:red">4</span> | <br>|<span style="color:green">3</span> | <span style="color:red">1</span> | <span style="color:red">6</span> | <br>

Solution found with IDDFS: [2, 7, 5, 4, 8, 1, 6, 8, 4, 5, 1, 2, 7, 1, 2, 7, 3, 6, 7, 4, 5] with a length of : 21
Execution time : 5.509913206100464 seconds

Solution not found with Manhatten heuristic in a 200 move limit
Execution time : 0.01874089241027832 seconds

Solution not found with Hamming heuristic in a 200 move limit
Execution time : 0.01796579360961914 seconds

Solution not found with Custom heuristic in a 200 move limit
Execution time : 0.020091772079467773 seconds


A* stategy

In [72]:
def matrix_to_tuple(matrix):
    return tuple(tuple(row) for row in matrix)

def tuple_to_matrix(tuples):
    return list(list(row) for row in tuples)

def dif_between_matrix(prev, current):
    for i in range(3):
        for j in range(3):
            if prev[i][j][0] != current[i][j][0] and current[i][j][0] != 0:
                return current[i][j][0]
    return 0

class State:
    def __init__(self, state, depth):
        self.state = state
        self.depth = depth

    def expand(self):
        successors = []
        m = tuple_to_matrix(self.state)
        for cell in [m[i][j][0] for i in range(3) for j in range(3) if m[i][j][1]]:
            moveMatrix = tranzitie(m, cell)
            successors.append(State(matrix_to_tuple(moveMatrix), self.depth + 1))
        return successors
    
    def __lt__(self, other):
        return (manhattan_distance(self.state)) < (manhattan_distance(other.state))

def final_proccesing(state_list):
    filter_steps = []
    depth_list = []
    moves = []
    index = []
    for i in state_list:
        depth_list.append(i.depth)
    for i in range(1, state_list[len(state_list) - 1].depth-1):
        l_index = max([j for j in depth_list if depth_list[j] == i] + [-1])
        index.append(l_index)

    for i in index:
        filter_steps.append(tuple_to_matrix(state_list[i].state))

    for i in range(1, len(filter_steps)):
        #print_game(filter_steps[i])
        moves.append(dif_between_matrix(filter_steps[i-1], filter_steps[i]))
    return moves

def A_star_search(initial_state):
    state_queue = PriorityQueue()
    state_queue.put((State(matrix_to_tuple(initial_state), 1)))
    closed_list = set()
    steps = [State(matrix_to_tuple(initial_state), 1)]

    while not state_queue.empty():
        current_node = state_queue.get()
        steps.append(current_node)
        if verificare_stare_finala(matrix_to_tuple(current_node.state)):
            return list(filter(lambda x: x != 0, final_proccesing(steps)))

        closed_list.add(current_node)

        for neighbor_node in current_node.expand():
            neighbor_state_tuple = matrix_to_tuple(neighbor_node.state)
            if neighbor_state_tuple not in closed_list:
                state_queue.put((neighbor_node))


    return None

In [73]:

stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
print_game(matrix)
print(A_star_search(matrix))
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
print_game(matrix)
print(A_star_search(matrix))

stateList = [8, 6, 7, 2, 5, 4, 0, 3, 1]
matrix = model_problem(stateList)
print_game(matrix)
print(A_star_search(matrix))


|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

[7, 5, 2]


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

[4, 3, 6, 8, 7, 4, 5, 2]


|<span style="color:red">8</span> | <span style="color:red">6</span> | <span style="color:red">7</span> | <br>|<span style="color:green">2</span> | <span style="color:red">5</span> | <span style="color:red">4</span> | <br>|0 | <span style="color:green">3</span> | <span style="color:red">1</span> | <br>

[3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 6, 8, 2, 3, 1, 4, 7, 5, 8, 6, 5, 8, 6, 2, 3, 3, 6, 2, 3, 6, 2, 2]


In [76]:
#adugam A* in 6:
def all_strategies(matrix):
    print_game(matrix)
    start_time = time.time()
    rez_IDDFS = IDDFS_search(matrix, 40)
    end_time = time.time()
    execution_time_IDDFS = end_time - start_time
    if(rez_IDDFS == "Limita de pasi atinsa"):
        print("Solution not found with IDDFS in a 40 move limit")
    else:
        print("Solution found with IDDFS: " + str(rez_IDDFS) + " with a length of : " + str(len(rez_IDDFS)))
    print("Execution time : " + str(execution_time_IDDFS) + " seconds")

    print()
    
    start_time = time.time()
    rez_Manhatten = greedy_manhattan(matrix, 200)
    end_time = time.time()
    execution_time_Manhatten = end_time - start_time
    if(rez_Manhatten == "Limita de pasi atinsa"):
        print("Solution not found with Manhatten heuristic in a 200 move limit")
    else:
        print("Solution found with Manhatten heuristic: " + str(rez_Manhatten) + " with a length of : " + str(len(rez_Manhatten)))
    print("Execution time : " + str(execution_time_Manhatten) + " seconds")

    print()

    start_time = time.time()
    rez_Hamming = greedy_hamming(matrix, 200)
    end_time = time.time()
    execution_time_Hamming = end_time - start_time
    if(rez_Hamming == "Limita de pasi atinsa"):
        print("Solution not found with Hamming heuristic in a 200 move limit")
    else:
        print("Solution found with Hamming heuristic: " + str(rez_Hamming) + " with a length  of : " + str(len(rez_Hamming)))
    print("Execution time : " + str(execution_time_Hamming) + " seconds")

    print()

    start_time = time.time()
    rez_Custom = greedy_euclidean(matrix, 200)
    end_time = time.time()
    execution_time_Custom = end_time - start_time
    if(rez_Custom == "Limita de pasi atinsa"):
        print("Solution not found with Custom heuristic in a 200 move limit")
    else:
        print("Solution found with Custom heuristic: " + str(rez_Custom) + " with a length of : " + str(len(rez_Custom)))
    print("Execution time : " + str(execution_time_Custom) + " seconds")

    print()

    start_time = time.time()
    rez_A = A_star_search(matrix)
    end_time = time.time()
    execution_time_A = end_time - start_time
    print("Solution found with A*: " + str(rez_A) + " with a length of : " + str(len(rez_A)))
    print("Execution time : " + str(execution_time_A) + " seconds")

In [77]:
stateList = [2, 5, 3, 1, 7, 6, 4, 0, 8]
matrix = model_problem(stateList)
all_strategies(matrix)
 
stateList = [1, 5, 2, 7, 0, 4, 8, 6, 3]
matrix = model_problem(stateList)
all_strategies(matrix)

|<span style="color:red">2</span> | <span style="color:red">5</span> | <span style="color:red">3</span> | <br>|<span style="color:red">1</span> | <span style="color:green">7</span> | <span style="color:red">6</span> | <br>|<span style="color:green">4</span> | 0 | <span style="color:green">8</span> | <br>

Solution found with IDDFS: [7, 5, 2, 1, 4] with a length of : 5
Execution time : 0.0010058879852294922 seconds

Solution found with Manhatten heuristic: [7, 5, 2, 1, 4] with a length of : 5
Execution time : 0.000988006591796875 seconds

Solution not found with Hamming heuristic in a 200 move limit
Execution time : 0.01900172233581543 seconds

Solution not found with Custom heuristic in a 200 move limit
Execution time : 0.02602386474609375 seconds

Solution found with A*: [7, 5, 2] with a length of : 3
Execution time : 0.001004934310913086 seconds


|<span style="color:red">1</span> | <span style="color:green">5</span> | <span style="color:red">2</span> | <br>|<span style="color:green">7</span> | 0 | <span style="color:green">4</span> | <br>|<span style="color:red">8</span> | <span style="color:green">6</span> | <span style="color:red">3</span> | <br>

Solution found with IDDFS: [4, 3, 6, 8, 7, 4, 5, 2, 3, 6] with a length of : 10
Execution time : 0.036635398864746094 seconds

Solution found with Manhatten heuristic: [5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5] with a length of : 19
Execution time : 0.001999378204345703 seconds

Solution found with Hamming heuristic: [5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5] with a length  of : 19
Execution time : 0.0019981861114501953 seconds

Solution found with Custom heuristic: [5, 2, 4, 3, 6, 8, 7, 1, 2, 4, 3, 6, 8, 5, 4, 2, 1, 4, 5] with a length of : 19
Execution time : 0.0019974708557128906 seconds

Solution found with A*: [4, 3, 6, 8, 7, 4, 5, 2] with a length of : 8
Execution time : 0.001003265380859375 seconds
